In [292]:
# Loading data

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
DF=pd.concat([train,test])
pd.options.display.max_columns=30

In [293]:
# Decoding names
def decodeTitle(name):
    signif_titles=['Mr.','Miss.','Mrs.','Master.'] 
    # cut exactly title from column Name
    title=name[name.find(',')+2:name.find('.')+1]
    if title in signif_titles:  return(title)
    else: return('Arist.')
    
DF['Title']=DF['Name'].apply(decodeTitle)
DF.loc[88,'Title']='Miss.'
DF.loc[550,'Name']='Thayer, Mr. NOjo Borland Jr'

In [332]:
# Decoding cabins
def number(string):
    numbers=[int(i) for i in re.findall('(\d+)',string)]
    return(np.mean(numbers))
def symbol(string):
    if pd.notnull(string):
        return(True)
    else: return(False)
DF['CabinSymbol'] = DF.Cabin.apply(symbol)
DF['CabinNumber'] = DF.Cabin.apply(number)

TypeError: expected string or bytes-like object

In [70]:
median_age_DF=DF.groupby(['Title','Pclass'])['Age'].median()
median_fare_DF=DF.groupby(['Title','Pclass'])['Fare'].median()
DF['Age']= DF.apply(lambda row: median_age_DF[row['Title'],row['Pclass']] if np.isnan(row['Age'])  else row['Age'],axis=1)
DF['Fare']=DF.apply(lambda row:median_fare_DF[row['Title'],row['Pclass']] if np.isnan(row['Fare']) else row['Fare'],axis=1)

In [308]:
def name(name):
        name=name[name.find('. ')+2:]
        if name.find(' ')==-1:
            return(name)
        else: return(name[:name.find(' ')])
def virgin_surname(name):
    if name.find('(')!=-1:
        return(name[name.rfind(' '):name.find(')')])
def husb_or_wife(row):
    if (row[8]=='female') and (row[4].find('Mrs.')!=-1):
        return(row[17])
DF['Surname']=DF.Name.apply(lambda x: x[:x.find(',')])
DF['Virg_surname']=DF.Name.apply(virgin_surname)
DF['First_name']=DF.Name.apply(name)
DF['Husb_name']=DF.apply(husb_or_wife,axis=1)

In [309]:
DF['wife']=DF.apply(find_my_wife, axis=1)
DF['how_is_wife']=DF.apply(how_is_wife, axis=1)
DF['childs']=DF.apply(find_my_childs, axis=1)
DF['bro']=DF.apply(find_my_bro, axis=1)
DF['how_are_childs']=DF.apply(how_are_childs, axis=1)    
DF['how_are_bro']=DF.apply(how_are_bro,axis=1)
DF['parents']=DF.apply(find_my_parents, axis=1)
DF['how_are_parents']=DF.apply(how_are_parents, axis=1) 

C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:46: RuntimeWarning: Mean of empty slice
C:\Users\user\Anaconda3\lib\site-packages\ipykernel\__main__.py:59: RuntimeWarning: Mean of empty slice


[451, 473]
451 450    0.0
Name: Survived, dtype: float64
473 472    1.0
Name: Survived, dtype: float64


In [73]:
DF['Title_code']=DF['Title'].astype('category').cat.codes
DF['CabinSymbol']=DF['CabinSymbol'].astype('category').cat.codes
DF['Sex']=DF['Sex'].astype('category').cat.codes
DF['Embarked']=DF['Embarked'].astype('category').cat.codes

In [ ]:
from sklearn.utils import shuffle
DF = shuffle(DF, random_state = 187)

In [254]:
DF_train=DF[DF.Survived.notnull()]
DF_test=DF[DF.Survived.notnull()==False]
# X_train=DF_train[['Family_param','Pclass','Sex','Title_code']]
# X_train.fillna(-1, inplace=True)
# Y_train=DF_train['Survived']
# from sklearn.grid_search import GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import cross_val_score
# optimizer = GridSearchCV(RandomForestClassifier(), \
#                          {'max_features' : [0.4, 0.6, 0.8, 1.0], 
#                           'max_depth' : [5, 10, 15, 25, 35, 45, 55, 65], 
#                           'n_estimators' : [1, 5, 10, 15, 30, 50, 70, 90, 100]},\
#                          cv = 15, \
#                          n_jobs=-1)
# optimizer.fit(X_train,Y_train)
# print('Random Forest Best Result', optimizer.best_score_)
# print('Best Parameters', optimizer.best_params_)

In [ ]:
X_test=DF_test[['Family_param','Pclass','Sex','Title_code']]
def writeAnswer(filename, optimizer):
    prediction = optimizer.predict(X_test)
    submission = pd.DataFrame({
            "PassengerId": test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.to_csv(filename, index=False)
    
writeAnswer('results.csv', optimizer)

In [312]:
X_train=DF_train[['Family_param']]
Y_train=DF_train['Survived']
X_test=DF_test[['Family_param']]
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, Y_train)
print(cross_val_score(model,X_train, Y_train))

def writeAnswer(filename, model):
    prediction = model.predict(X_test)
    submission = pd.DataFrame({
            "PassengerId": DF_test["PassengerId"],
            "Survived": prediction
        })
    submission.Survived=submission.Survived.astype('int64',copy=False)
    submission.sort_values('PassengerId').to_csv(filename, index=False)
    return(submission)
    
r=writeAnswer('results.csv', model)

[ 0.85185185  0.86195286  0.83164983]


In [306]:
## Big pack of setting of relationship between passengers
def find_my_wife(row):
    if (row[8]=='male') and (row[12]!='Master.') and ('Jr' not in row[4]):
        probably_wife=DF[(DF['Surname']==row[15]) & (DF['Husb_name']==row[17]) & (DF['Title']=='Mrs.') & (DF['Title']!='Arist.')]
        if len(probably_wife)!=0:
            return(int(probably_wife['PassengerId']))
        else:return(-999)
    else: return(-888)

#0 - not survived, 1- survived, 2 - not in a board, 3 - woman or child, 4 - no data
def how_is_wife(row):
    if row[19]>0:
        surv=DF[DF['PassengerId']==row[19]]['Survived']
        if pd.notnull(surv.values):
            return(surv.values[0])
        else: return(4)
    elif row[19]>-900: return(3)
    else: return(2)


def find_my_childs(row):
    if row[12] in ['Mrs.','Mr.','Arist.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Master.') | (DF['Title']=='Miss.') | (DF['Title']=='Mr.'))]
        probably_childs=probably_childs[(probably_childs['SibSp']==row[5]-1) & (probably_childs['Name']!=row[4])]
        probably_childs=probably_childs[(probably_childs['wife']<0) | (probably_childs['SibSp']!=1)]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def find_my_bro(row):
    if row[12] in ['Master.','Miss.','Mr.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Master.') | (DF['Title']=='Miss.') | (DF['Title']=='Mr.'))]
        probably_childs=probably_childs[(probably_childs['Parch']==row[5]) & (probably_childs['SibSp']==row[9]) & (probably_childs['Name']!=row[4])]
        probably_childs=probably_childs[probably_childs['PassengerId'].isin(row[21])==False]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def how_are_childs(row):
    if row[21][0]>0:
        surv_index,j=[],np.nan
        for i in row[21]:
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
            j=np.nanmean(surv_index)
        if pd.notnull(j):
            return(j)
        else:return(4)
    elif row[21][0]>-900: return(3)
    else: return(2)   
    
    
def how_are_bro(row):
    if row[22][0]>0:
        surv_index,j=[],np.nan
        for i in row[22]:
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
            j=np.nanmean(surv_index)
        if pd.notnull(j):
            return(j)
        else:return(4)
    elif row[22][0]>-900: return(3)
    else: return(2)   
    

def find_my_parents(row):
    if row[12] in ['Miss.','Master.']:
        probably_childs=DF[(DF['Surname']==row[15]) & ((DF['Title']=='Mrs.') | (DF['Title']=='Mr.') | (DF['Title']=='Arist.'))]
        probably_childs=probably_childs[(probably_childs['SibSp']==row[5]-1) & (probably_childs['Name']!=row[4])]
        if len(probably_childs)!=0:
            return(list(probably_childs['PassengerId']))
        else: return([-999,0])
    else: return([-888,0])

def how_are_parents(row):
    if row[25][0]>0:
        if row[6]==59: print(row[25])
        surv_index,j=[],np.nan
        for i in row[25]:
            if row[6]==59: print(i, DF[DF['PassengerId']==i]['Survived'])
            surv_index.append(DF[DF['PassengerId']==i]['Survived'].iloc[0])
        for q in surv_index: 
            if pd.notnull(q)==False:return(4)
        j=np.nanmean(surv_index)
        if pd.notnull(j):
            return(j)
        else:return(4)
    elif row[25][0]>-900: return(3)
    else: return(2)  

In [311]:
def My_family(row):
    if row[12] in ['Miss.','Master.']:
        if row['how_are_parents']==0 or (row['how_are_bro']==0 and row['Pclass']==3): return(0)
        else:return(1)
    if row['Title'] in ['Mrs.','Arist.'] and row['Sex']==0:
        if row['how_are_child']<0.5: return(0)
        else: return(1)
    if row['Title'] in ['Mr.','Arist.'] and row['Sex']==1:
        return(0)
            
    
    
DF['Family_param']=DF.apply(My_family,axis=1)

In [75]:
def tick_number(string):
    tick=re.findall('(\d+)',string)
    if len(tick)>0:
        return(int(tick[-1]))
    else: return(999999)
DF['Ticket_number']=DF.Ticket.apply(tick_number)

In [328]:
DF['Ticket_symbol']=DF['Ticket'].apply(lambda s: re.sub('[^a-zA-Z]+', '', s))

In [201]:
DF=DF.sort_values(['Ticket_symbol','Ticket_number'])
DF['Ticket_of_my_neight']=DF['Ticket_number'].diff()
group=0
def my_group(row):
    global group
    if np.abs(row[34])>5: group+=1
    return(group) 
DF['my_group']=DF.apply(my_group,axis=1)

In [197]:
def how_are_neight(x):
    return(np.nanmean(DF[(DF['group']==x) & (DF['Title']=='Mr.')]['Survived'].values))
DF['my_neight']=DF.group.apply(how_are_neight)

,Age,Cabin,Embarked,Fare,Name,Parch,PassengerId,Pclass,Sex,SibSp,Survived,Ticket,Title,CabinSymbol,CabinNumber,...,Virg_surname,First_name,Husb_name,wife,how_is_wife,childs,bro,how_are_childs,how_are_bro,parents,how_are_parents,husband,how_is_my_husb,Family_param,Ticket_symbol
0,22.0,NaN,S,7.2500,"Braund, Mr. Owen Harris",0,1,3,male,1,0.0,A/5 21171,Mr.,False,NaN,...,None,Owen,None,-999,2.0,"[-999, 0]",[478],2.000000,0.0,"[-888, 0]",3.0,-888,3.0,NaN,A
1,38.0,C85,C,71.2833,"Cumings, Mrs. John Bradley (Florence Briggs Th...",0,2,1,female,1,1.0,PC 17599,Mrs.,True,85.0,...,Thayer,John,John,-888,3.0,"[-999, 0]","[-888, 0]",2.000000,3.0,"[-888, 0]",3.0,1126,4.0,NaN,PC
2,26.0,NaN,S,7.9250,"Heikkinen, Miss. Laina",0,3,3,female,0,1.0,STON/O2. 3101282,Miss.,False,NaN,...,None,Laina,None,-888,3.0,"[-888, 0]","[-999, 0]",3.000000,2.0,"[-999, 0]",2.0,-888,3.0,1.0,STONO
3,35.0,C123,S,53.1000,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",0,4,1,female,1,1.0,113803,Mrs.,True,123.0,...,Peel,Jacques,Jacques,-888,3.0,"[-999, 0]","[-888, 0]",2.000000,3.0,"[-888, 0]",3.0,138,0.0,NaN,
4,35.0,NaN,S,8.0500,"Allen, Mr. William Henry",0,5,3,male,0,0.0,373450,Mr.,False,NaN,...,None,William,None,-999,2.0,"[-999, 0]",[731],2.000000,1.0,"[-888, 0]",3.0,-888,3.0,NaN,
5,26.0,NaN,Q,8.4583,"Moran, Mr. James",0,6,3,male,0,0.0,330877,Mr.,False,NaN,...,None,James,None,-999,2.0,"[-999, 0]","[-999, 0]",2.000000,2.0,"[-888, 0]",3.0,-888,3.0,NaN,
6,54.0,E46,S,51.8625,"McCarthy, Mr. Timothy J",0,7,1,male,0,0.0,17463,Mr.,True,46.0,...,None,Timothy,None,-999,2.0,"[-999, 0]",[1196],2.000000,4.0,"[-888, 0]",3.0,-888,3.0,NaN,
7,2.0,NaN,S,21.0750,"Palsson, Master. Gosta Leonard",1,8,3,male,3,0.0,349909,Master.,False,NaN,...,None,Gosta,None,-888,3.0,"[-888, 0]","[25, 375, 1281]",3.000000,0.0,[568],0.0,-888,3.0,0.0,
8,27.0,NaN,S,11.1333,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",2,9,3,female,0,1.0,347742,Mrs.,False,NaN,...,Berg,Oscar,Oscar,-888,3.0,"[173, 870]","[-888, 0]",1.000000,3.0,"[-888, 0]",3.0,-999,2.0,NaN,
9,14.0,NaN,C,30.0708,"Nasser, Mrs. Nicholas (Adele Achem)",0,10,2,female,1,1.0,237736,Mrs.,False,NaN,...,Achem,Nicholas,Nicholas,-888,3.0,"[-999, 0]","[-888, 0]",2.000000,3.0,"[-888, 0]",3.0,123,0.0,NaN,
